In [1]:
from pathlib import Path
import pandas as pd
import mne
import numpy as np
from HFODetector import ste  # or mni, if you prefer

In [2]:
# 1) instantiate your detector once
detector = ste.STEDetector(
    sample_freq=2000,
    filter_freq=[80,500],
    rms_window=3e-3,
    min_window=6e-3,
    min_gap=10e-3,
    epoch_len=600,
    min_osc=6,
    rms_thres=5,
    peak_thres=3,
    n_jobs=8
)

results = []

In [3]:
# 2) point to your top‐level BIDS folder
data_root = Path("data/ds004100")

# Sanity Check
if data_root.exists():
        print("Path exists.")
else:
        print("Path does not exist.")

Path exists.


In [4]:
for edf_path in data_root.rglob("*_task-ictal_*_ieeg.edf"):
    # skip any placeholder/pointer files
    if not edf_path.is_file():
        print(f"Missing on disk, skipping: {edf_path}")
        continue

    # 3) Load the channels.tsv to find which channels are GOOD
    ch_path = edf_path.with_name(edf_path.name.replace("_ieeg.edf", "_channels.tsv"))
    ch_df   = pd.read_csv(ch_path, sep="\t")
    good_chs = ch_df.loc[ch_df.status != "bad", "name"].tolist()

    # 4) Run the detector directly on the EDF
    chns, se = detector.detect_edf(str(edf_path))

    # 5) Flatten into a list of event‐dicts (handles zero events gracefully)
    records = [
        {
            "channel": ch_name,
            "start_s": start_idx / detector.sample_freq,
            "end_s":   end_idx   / detector.sample_freq
        }
        for ch_name, chan_events in zip(chns, se)
        for start_idx, end_idx in chan_events
    ]
    hfo_df = pd.DataFrame.from_records(records, columns=["channel","start_s","end_s"])

    # 6) Drop any HFOs on “bad” channels
    hfo_df = hfo_df[hfo_df.channel.isin(good_chs)]

    # 7) Summarize count & rate
    n_hfos = len(hfo_df)
    raw    = mne.io.read_raw_edf(str(edf_path), preload=False)
    duration_s = raw.n_times / raw.info["sfreq"]
    hfo_rate   = n_hfos / duration_s if duration_s > 0 else np.nan

    # 8) Extract subject/run IDs
    subj = edf_path.parents[2].name   # == "sub-HUP060"               # e.g. "sub-HUP060"
    run  = edf_path.stem.split("_")[-2]        # e.g. "run-01"

    results.append({
        "subject":    subj,
        "run":        run,
        "n_hfos":     n_hfos,
        "duration_s": duration_s,
        "hfo_rate":   hfo_rate
    })

Extracting EDF parameters from /Users/billhuang/Desktop/BU_RISE/BU_RISE_Project/data/ds004100/sub-HUP089/ses-presurgery/ieeg/sub-HUP089_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


100%|██████████| 97.0/97.0 [00:02<00:00, 40.5it/s]


Extracting EDF parameters from /Users/billhuang/Desktop/BU_RISE/BU_RISE_Project/data/ds004100/sub-HUP089/ses-presurgery/ieeg/sub-HUP089_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/billhuang/Desktop/BU_RISE/BU_RISE_Project/data/ds004100/sub-HUP089/ses-presurgery/ieeg/sub-HUP089_ses-presurgery_task-ictal_acq-ecog_run-02_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


100%|██████████| 97.0/97.0 [00:02<00:00, 40.9it/s]


Extracting EDF parameters from /Users/billhuang/Desktop/BU_RISE/BU_RISE_Project/data/ds004100/sub-HUP089/ses-presurgery/ieeg/sub-HUP089_ses-presurgery_task-ictal_acq-ecog_run-02_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/billhuang/Desktop/BU_RISE/BU_RISE_Project/data/ds004100/sub-HUP089/ses-presurgery/ieeg/sub-HUP089_ses-presurgery_task-ictal_acq-ecog_run-03_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

Extracting EDF parameters from /Users/billhuang/Desktop/BU_RISE/BU_RISE_Project/data/ds004100/sub-HUP097/ses-presurgery/ieeg/sub-HUP097_ses-presurgery_task-ictal_acq-ecog_run-03_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/billhuang/Desktop/BU_RISE/BU_RISE_Project/data/ds004100/sub-HUP097/ses-presurgery/ieeg/sub-HUP097_ses-presurgery_task-ictal_acq-ecog_run-02_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...



100%|██████████| 101/101 [00:04<00:00, 23.1it/s] 

Extracting EDF parameters from /Users/billhuang/Desktop/BU_RISE/BU_RISE_Project/data/ds004100/sub-HUP097/ses-presurgery/ieeg/sub-HUP097_ses-presurgery_task-ictal_acq-ecog_run-02_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /Users/billhuang/Desktop/BU_RISE/BU_RISE_Project/data/ds004100/sub-HUP064/ses-presurgery/ieeg/sub-HUP064_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...



100%|██████████| 93.0/93.0 [00:02<00:00, 41.8it/s]


Extracting EDF parameters from /Users/billhuang/Desktop/BU_RISE/BU_RISE_Project/data/ds004100/sub-HUP064/ses-presurgery/ieeg/sub-HUP064_ses-presurgery_task-ictal_acq-ecog_run-01_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [1]:
# 4) assemble everything into a single DataFrame
all_runs = pd.DataFrame(results)
all_runs.head()

NameError: name 'pd' is not defined

In [9]:
all_runs.to_csv('all_runs.csv', index=False) 